In [2]:
import cv2
import os

def detect_and_save_faces(image_dir, output_dir):
    # Load OpenCV's pre-trained face detection model
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through each person's folder
    for person in os.listdir(image_dir):
        person_dir = os.path.join(image_dir, person)
        if not os.path.isdir(person_dir):
            continue

        person_output_dir = os.path.join(output_dir, person)
        if not os.path.exists(person_output_dir):
            os.makedirs(person_output_dir)

        # Loop through each image in the person's folder
        for image_name in os.listdir(person_dir):
            img_path = os.path.join(person_dir, image_name)
            img = cv2.imread(img_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.1, 4)

            # Save the detected faces
            for (x, y, w, h) in faces:
                face_img = img[y:y+h, x:x+w]
                face_img_path = os.path.join(person_output_dir, image_name)
                cv2.imwrite(face_img_path, face_img)

# Specify your directories
image_dir = r"C:\Users\mitpa\OneDrive\Desktop\hrs\face detection project\DataSet Sept-Dec2023\data attendence 031023\Photoes"

output_dir = 'output_faces'

# Detect and save faces
detect_and_save_faces(image_dir, output_dir)


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Model training

In [4]:
# Load the DenseNet121 model without the top layer
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [5]:
# Add custom layers on top of DenseNet121
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(40, activation='softmax')(x)  # 40 classes for 40 folders

In [6]:
model = Model(inputs=base_model.input, outputs=predictions)


In [7]:
# Freeze the base model layers so they are not updated during training
for layer in base_model.layers:
    layer.trainable = False

In [8]:
# Compile the model with an optimizer and loss function
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [9]:
# Set up data augmentation and data generator
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [10]:
# Define training and validation data generators
train_generator = train_datagen.flow_from_directory(
    'output_faces',  # Path to the pre-processed face images
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 360 images belonging to 40 classes.


In [11]:
validation_generator = train_datagen.flow_from_directory(
    'output_faces',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 72 images belonging to 40 classes.


In [13]:
# Train the model
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=25,
    steps_per_epoch=train_generator.samples // 32,
    validation_steps=validation_generator.samples // 32
)

Epoch 1/25
11/11 [==============================] - 13s 1s/step - loss: 2.0493 - accuracy: 0.4787 - val_loss: 2.8859 - val_accuracy: 0.2812
Epoch 2/25
11/11 [==============================] - 12s 1s/step - loss: 1.9224 - accuracy: 0.5183 - val_loss: 2.8238 - val_accuracy: 0.3594
Epoch 3/25
11/11 [==============================] - 13s 1s/step - loss: 1.9405 - accuracy: 0.5122 - val_loss: 2.8732 - val_accuracy: 0.3438
Epoch 4/25
11/11 [==============================] - 13s 1s/step - loss: 1.7726 - accuracy: 0.5457 - val_loss: 2.8009 - val_accuracy: 0.3594
Epoch 5/25
11/11 [==============================] - 13s 1s/step - loss: 1.5932 - accuracy: 0.6037 - val_loss: 2.6695 - val_accuracy: 0.3750
Epoch 6/25
11/11 [==============================] - 13s 1s/step - loss: 1.5990 - accuracy: 0.5579 - val_loss: 2.9191 - val_accuracy: 0.3438
Epoch 7/25
11/11 [==============================] - 12s 1s/step - loss: 1.4964 - accuracy: 0.5976 - val_loss: 2.8861 - val_accuracy: 0.3906
Epoch 8/25
11/11 [==

In [14]:
# Save the trained model for future use
model.save('face_recognition_densenet121_labels.h5')

C:\Users\mitpa\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
import cv2
import numpy as np
import pandas as pd
import random
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from datetime import datetime

# Load the trained DenseNet121 model
model = load_model('face_recognition_densenet121_labels.h5')

# Define the labels (1 to 40) corresponding to the folders
labels = list(range(1, 41))

# Initialize an empty list to store attendance records
attendance = []

# Function to mark attendance
def mark_attendance(label):
    time_now = datetime.now()
    attendance.append({'Roll Number': label, 'Time': time_now.strftime('%H:%M:%S'), 'Date': time_now.strftime('%Y-%m-%d')})

# Function to save attendance to an Excel file
def save_attendance():
    df = pd.DataFrame(attendance)
    df.to_excel('attendance.xlsx', index=False)

# Function to check student presence in randomly selected segments
def check_student_presence(video_path, num_segments=10, threshold=7):
    video_capture = cv2.VideoCapture(video_path)
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))
    duration = total_frames / fps

    # Calculate segment length (in seconds)
    segment_length = duration / num_segments

    # Randomly select 10 segments from the video
    segments = sorted(random.sample(range(num_segments), num_segments))

    presence_count = 0
    recognized_label = None

    for segment in segments:
        # Set the video to the start of the segment
        start_frame = int(segment * segment_length * fps)
        video_capture.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

        ret, frame = video_capture.read()
        if not ret:
            continue

        # Convert the frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        # Check if the student's face is recognized
        for (x, y, w, h) in faces:
            face_img = frame[y:y+h, x:x+w]
            face_img = cv2.resize(face_img, (224, 224))
            face_array = img_to_array(face_img)
            face_array = np.expand_dims(face_array, axis=0) / 255.0

            # Predict the identity of the face
            predictions = model.predict(face_array)
            max_index = np.argmax(predictions[0])
            recognized_label = labels[max_index]

            # If face is recognized, increase presence count
            presence_count += 1
            break  # Only count the first recognized face in the segment

    # Determine if the student is present or absent
    if recognized_label is not None:
        if presence_count >= threshold:
            mark_attendance(recognized_label)
            print(f"Student with Roll Number {recognized_label} is Present")
        else:
            print(f"Student with Roll Number {recognized_label} is Absent")
    else:
        print("No recognizable face found in the selected segments.")

    video_capture.release()

# Run the presence check on your video
check_student_presence(r"C:\Users\mitpa\OneDrive\Desktop\hrs\face detection project\DataSet Sept-Dec2023\CCTV Footage 30_09_2023\22.9.23 class 1\172.16.116.247_Class_1_1_main_20230922130004_140004.asf")

# Save the attendance records when finished
save_attendance()


No recognizable face found in the selected segments.
